___
# Tags: #tools
___


In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

In [11]:
query_text = """ 
SELECT
    *
FROM db1.citizens_st_mobile_parquet
WHERE report_date = '2025-06-01'
limit 10
"""

ch.query_run(query_text)

,report_date,citizen_id,trial_available,state,flat_uuid,address_uuid
0,2025-06-01,1570953,0,registered,,
1,2025-06-01,1570954,0,activated,9e012c23-f9bf-4c13-b0f0-eb03a87f4650,1260194c-8f79-497d-8e9c-ce87903e1a91
2,2025-06-01,1570955,0,activated,c87f9825-01e8-4c1f-85c5-f44a7d15c927,eb1574da-219a-46a7-9628-799a43c3ef15
3,2025-06-01,1570956,0,activated,c05d10e6-72b1-4082-8fc7-851a4629fbd0,8edb9f03-d4b4-4c48-ab40-bb28f386af52
4,2025-06-01,1570957,0,registered,,
5,2025-06-01,1570958,0,registered,,
6,2025-06-01,1570959,0,activated,7305cbce-e34e-4150-b51e-f153df034aef,8dacd934-f218-41ca-af77-656bc339c640
7,2025-06-01,1570960,0,activated,8bae2299-2065-427f-b75e-f72c5b1255da,bdb95fb3-201f-4a93-9f47-1a9108d27ba0
8,2025-06-01,1570961,0,activated,800a6856-1f1c-4fb9-86f8-86b070442105,8edb9f03-d4b4-4c48-ab40-bb28f386af52
9,2025-06-01,1570962,0,activated,800a6856-1f1c-4fb9-86f8-86b070442105,8edb9f03-d4b4-4c48-ab40-bb28f386af52


# citizens_st_mobile_parquet remaking

In [ ]:
start_date = datetime.datetime.strptime('2023-07-10','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-05-29','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('%/year=%Y/month=%m/%d.csv')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
dates_pd

In [ ]:
start_date = datetime.datetime.strptime('2025-05-30','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-07-10','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('%/year=%-Y/month=%m/%d%')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
for day_index in dates_pd.index:
    date = dates_pd.loc[day_index,['date']].values[0]
    date_key = dates_pd.loc[day_index,['date_key']].values[0]
    query_text = f"""
        INSERT INTO db1.citizens_st_mobile_ch
        SETTINGS s3_truncate_on_insert = 1
        SELECT
            report_date,
            citizen_id,
            trial_available,
            state,
            activated_at,
            flat_uuid,
            address_uuid
        FROM db1.citizens_st_mobile
        WHERE report_date = '{date}'
            AND _path LIKE '{date_key}'
    """
    ch.query_run(query_text)
    print(date)

2025-05-30
2025-05-29
2025-05-28
2025-05-27
2025-05-26
2025-05-25
2025-05-24
2025-05-23
2025-05-22
2025-05-21
2025-05-20
2025-05-19
2025-05-18
2025-05-17
2025-05-16
2025-05-15
2025-05-14
2025-05-13
2025-05-12
2025-05-11
2025-05-10
2025-05-09
2025-05-08
2025-05-07
2025-05-06
2025-05-05
2025-05-04
2025-05-03
2025-05-02
2025-05-01
2025-04-30
2025-04-29
2025-04-28
2025-04-27
2025-04-26
2025-04-25
2025-04-24
2025-04-23
2025-04-22
2025-04-21
2025-04-20
2025-04-19
2025-04-18
2025-04-17
2025-04-16
2025-04-15
2025-04-14
2025-04-13
2025-04-12
2025-04-11
2025-04-10
2025-04-09
2025-04-08
2025-04-07
2025-04-06
2025-04-05
2025-04-04
2025-04-03
2025-04-02
2025-04-01
2025-03-31
2025-03-30
2025-03-29
2025-03-28
2025-03-27
2025-03-26
2025-03-25


KeyboardInterrupt: 

In [7]:
start_date = datetime.datetime.strptime('2023-07-10','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-05-29','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('%/year=%-Y/month=%m/%d%')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
for day_index in dates_pd.index:
    date = dates_pd.loc[day_index,['date']].values[0]
    date_key = dates_pd.loc[day_index,['date_key']].values[0]
    query_text = f"""
        INSERT INTO db1.citizens_st_mobile_ch
        SETTINGS s3_truncate_on_insert = 1
        SELECT
            citizens_st_mobile.report_date AS report_date,
            citizens_st_mobile.citizen_id AS citizen_id,
            citizens_st_mobile.trial_available AS trial_available,
            citizens_st_mobile.state AS state,
            citizens_st_mobile_ch_2025_05_30.activated_at AS activated_at,
            citizens_st_mobile_ch_2025_05_30.flat_uuid AS flat_uuid,
            citizens_st_mobile_ch_2025_05_30.address_uuid AS address_uuid
        FROM db1.citizens_st_mobile AS citizens_st_mobile
        LEFT JOIN
            (SELECT
                citizen_id,
                flat_uuid,
                address_uuid,
                activated_at
            FROM db1.citizens_st_mobile
            WHERE report_date = '2025-05-30'
             AND _path LIKE '{date_key}') AS citizens_st_mobile_ch_2025_05_30
            ON citizens_st_mobile_ch_2025_05_30.citizen_id = citizens_st_mobile.citizen_id
        WHERE report_date = '{date}'
            AND _path LIKE '{date_key}'
    """
    ch.query_run(query_text)
    print(date)
# ch.query_run(query_text)


2025-05-29
2025-05-28
2025-05-27
2025-05-26
2025-05-25
2025-05-24
2025-05-23
2025-05-22
2025-05-21
2025-05-20
2025-05-19
2025-05-18
2025-05-17
2025-05-16
2025-05-15
2025-05-14
2025-05-13
2025-05-12
2025-05-11
2025-05-10
2025-05-09
2025-05-08
2025-05-07
2025-05-06
2025-05-05
2025-05-04
2025-05-03
2025-05-02
2025-05-01
2025-04-30
2025-04-29
2025-04-28
2025-04-27
2025-04-26
2025-04-25
2025-04-24
2025-04-23
2025-04-22
2025-04-21
2025-04-20
2025-04-19
2025-04-18
2025-04-17
2025-04-16
2025-04-15
2025-04-14
2025-04-13
2025-04-12
2025-04-11
2025-04-10
2025-04-09
2025-04-08
2025-04-07
2025-04-06
2025-04-05
2025-04-04
2025-04-03
2025-04-02
2025-04-01
2025-03-31
2025-03-30
2025-03-29
2025-03-28
2025-03-27
2025-03-26
2025-03-25
2025-03-24
2025-03-23
2025-03-22
2025-03-21
2025-03-20
2025-03-19
2025-03-18
2025-03-17
2025-03-16
2025-03-15
2025-03-14
2025-03-13
2025-03-12
2025-03-11
2025-03-10
2025-03-09
2025-03-08
2025-03-07
2025-03-06
2025-03-05
2025-03-04
2025-03-03
2025-03-02
2025-03-01
2025-02-28

## Обновление данных после 2025-05-30

In [ ]:
start_date = datetime.datetime.strptime('2025-05-30','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-06-30','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.parquet')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
for day_index in dates_pd.index:
    date = dates_pd.loc[day_index,['date']].values[0]
    date_key = dates_pd.loc[day_index,['date_key']].values[0]
    query_text = f"""
        INSERT INTO FUNCTION s3(
        'https://storage.yandexcloud.net/dwh-asgard/citizens_st_mobile_parquet/{date_key}',
        'parquet'
        )
        SETTINGS s3_truncate_on_insert = 1
        SELECT
            report_date,
            citizen_id,
            trial_available,
            state,
            activated_at,
            flat_uuid,
            address_uuid
        FROM db1.citizens_st_mobile_ch
        WHERE report_date = '{date}'
        
    """
    ch.query_run(query_text)
    print(date)
# ch.query_run(query_text)


In [3]:
start_date = datetime.datetime.strptime('2025-07-04','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-07-04','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.parquet')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
for day_index in dates_pd.index:
    date = dates_pd.loc[day_index,['date']].values[0]
    date_key = dates_pd.loc[day_index,['date_key']].values[0]
    query_text = f"""
        INSERT INTO FUNCTION s3(
        'https://storage.yandexcloud.net/dwh-asgard/citizens_st_mobile_parquet/{date_key}',
        'parquet'
        )
        SETTINGS s3_truncate_on_insert = 1
        SELECT
            report_date,
            citizen_id,
            trial_available,
            state,
            activated_at,
            flat_uuid,
            address_uuid
        FROM db1.citizens_st_mobile_ch
        WHERE report_date = '{date}'
        
    """
    ch.query_run(query_text)
    print(date)
# ch.query_run(query_text)

2025-07-04


In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.citizens_st_mobile_parquet_mv
"""

ch.query_run(query_text)

citizens_st_mobile_parquet_mv

In [ ]:
query_text = """
SELECT
    *
FROM db1.citizens_st_mobile_parquet
WHERE flat_uuid != ''
ORDER BY report_date
limit 10
"""

ch.query_run(query_text)

In [ ]:
query_text = """
SELECT
    *
FROM db1.citizens_st_mobile_parquet
WHERE flat_uuid != ''
ORDER BY report_date
limit 10
"""

ch.query_run(query_text)

In [ ]:
query_text = """--sql
    CREATE TABLE db1.citizens_st_mobile_remake_parquet_ch
    (
    `report_date` Date,
    `citizen_id` Int32,
    `trial_available` Int32,
    `state` String,
    `flat_uuid` String,
    `address_uuid` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """

ch.query_run(query_text)

In [ ]:
query_text = """
CREATE MATERIALIZED VIEW db1.citizens_st_mobile_remake_parquet_mv
REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.citizens_st_mobile_remake_parquet_ch AS
INSERT INTO FUNCTION s3(
    'https://storage.yandexcloud.net/dwh-asgard/citizens_st_mobile_remake_parquet_ch/{date_key}',
    'parquet'
    )
    SETTINGS s3_truncate_on_insert = 1
    SELECT
        report_date,
        citizen_id,
        trial_available,
        state,
        flat_uuid,
        address_uuid
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    WHERE report_date = '{date}'
        
    """

ch.query_run(query_text)

# add date for billing_orders_devices_st_partner

In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet('/home/boris/Downloads/citizens_st_mobile_parquet.parquet')

In [8]:
query_text = """--sql 
    CREATE TABLE db1.billing_orders_devices_dir_partner
    (   
        `billing_account_id` Int64,
        `cost` Float64,
        `count` Int64,
        `created_at` DateTime,
        `device_type` String,
        `device_uuid` String,
        `partner_uuid` String,
        `report_date` Date,
        `service` String,
        `state` String,
        `total` Float64
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/billing_orders_devices_dir_partner/*.csv','CSVWithNames')
    PARTITION BY billing_account_id
    """

ch.query_run(query_text)

""


In [8]:
import pandas as pd
df = pd.read_parquet('/home/boris/Downloads/29.parquet')

df[df['flat_uuid'].isna() == True]

,report_date,citizen_id,trial_available,state,flat_uuid,address_uuid


In [7]:
query_text = """
    DROP TABLE db1.billing_orders_devices_dir_partner
    """
ch.query_run(query_text)

,elapsed_ns,query_id
0,1280404,a4d921ab-84c6-4918-a689-c4cd128c22d2


In [21]:
query_text = """--sql
    SELECT
        *
    FROM db1.billing_orders_devices_st_partner
    WHERE report_date = '2025-05-22'
    limit 10
    """

ch.query_run(query_text)

,billing_account_id,cost,count,created_at,device_type,device_uuid,partner_uuid,report_date,service,state,total
0,18,15.0,1,2025-05-22 00:00:31,intercom,0012bdf9-9813-4e3e-b6b4-1df7d8eea95d,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-22,archive_3_kz,success,15.0
1,18,15.0,1,2025-05-22 00:00:31,camera,003d8f66-31ae-4561-92e5-2b21c7930a15,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-22,archive_3_kz,success,15.0
2,18,15.0,1,2025-05-22 00:00:31,intercom,029ee312-ea6f-4adb-a68a-b88d27a774aa,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-22,archive_3_kz,success,15.0
3,18,15.0,1,2025-05-22 00:00:31,intercom,03d24162-0350-44aa-94ec-76f04ecf37c8,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-22,archive_3_kz,success,15.0
4,18,15.0,1,2025-05-22 00:00:31,intercom,03f89b86-1303-451b-b0ec-6cfc408e8fe9,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-22,archive_3_kz,success,15.0
5,18,15.0,1,2025-05-22 00:00:31,intercom,048e2669-8ec5-48f8-a0eb-7145e032c0d8,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-22,archive_3_kz,success,15.0
6,18,15.0,1,2025-05-22 00:00:31,intercom,049c963f-3641-4e2a-9eb6-e3ad178cd651,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-22,archive_3_kz,success,15.0
7,78,2.8,1,2025-05-22 00:00:31,intercom,707c6373-dba9-4e1e-b5ad-3bcd8f6edb2c,fdae9241-37f2-4446-866c-cd9119425fe4,2025-05-22,archive_3_pro,success,2.8
8,18,15.0,1,2025-05-22 00:00:31,camera,055f53e8-2858-421a-b736-bc1cfb2231b5,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-22,archive_3_kz,success,15.0
9,78,4.9,1,2025-05-22 00:00:31,intercom,e89eeede-6c16-455a-bbbf-31823c2db007,fdae9241-37f2-4446-866c-cd9119425fe4,2025-05-22,archive_7_pro,success,4.9


In [20]:
query_text = """--sql
    SELECT
        *
    FROM db1.billing_orders_devices_st_partner
    WHERE report_date = '2025-05-21'
    limit 10
    """

ch.query_run(query_text)

,billing_account_id,cost,count,created_at,device_type,device_uuid,partner_uuid,report_date,service,state,total
0,18,15.0,1,2025-05-21 00:00:14,intercom,0012bdf9-9813-4e3e-b6b4-1df7d8eea95d,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-21,archive_3_kz,success,15.0
1,18,15.0,1,2025-05-21 00:00:14,camera,003d8f66-31ae-4561-92e5-2b21c7930a15,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-21,archive_3_kz,success,15.0
2,18,15.0,1,2025-05-21 00:00:14,intercom,029ee312-ea6f-4adb-a68a-b88d27a774aa,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-21,archive_3_kz,success,15.0
3,18,15.0,1,2025-05-21 00:00:14,intercom,03d24162-0350-44aa-94ec-76f04ecf37c8,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-21,archive_3_kz,success,15.0
4,18,15.0,1,2025-05-21 00:00:14,intercom,03f89b86-1303-451b-b0ec-6cfc408e8fe9,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-21,archive_3_kz,success,15.0
5,18,15.0,1,2025-05-21 00:00:14,intercom,048e2669-8ec5-48f8-a0eb-7145e032c0d8,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-21,archive_3_kz,success,15.0
6,18,15.0,1,2025-05-21 00:00:14,intercom,049c963f-3641-4e2a-9eb6-e3ad178cd651,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-21,archive_3_kz,success,15.0
7,18,15.0,1,2025-05-21 00:00:15,camera,055f53e8-2858-421a-b736-bc1cfb2231b5,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-21,archive_3_kz,success,15.0
8,18,15.0,1,2025-05-21 00:00:15,camera,05f95ce5-90a7-4c0d-a7a7-d3cca406cae5,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-21,archive_3_kz,success,15.0
9,18,15.0,1,2025-05-21 00:00:15,camera,0711124e-0f71-41f9-8d8c-c5a8e3c309af,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-05-21,archive_3_kz,success,15.0


In [23]:
start_date = datetime.datetime.strptime('2025-01-24','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-05-21','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%-Y/month=%-m/%-d.csv')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
for day_index in dates_pd.index:
    date = dates_pd.loc[day_index,['date']].values[0]
    date_key = dates_pd.loc[day_index,['date_key']].values[0]
    query_text = f"""
        INSERT INTO FUNCTION s3(
        'https://storage.yandexcloud.net/dwh-asgard/billing_orders_devices_st_partner/{date_key}',
        'CSVWithNames'
        )
        SETTINGS s3_truncate_on_insert = 1
            SELECT
            `billing_account_id`,
                `cost`,
                `count`,
                `created_at`,
                `device_type`,
                `device_uuid`,
                `partner_uuid`,
                toDate(created_at) AS `report_date`,
                `service`,
                `state`,
                `total`

            FROM db1.billing_orders_devices_dir_partner
        WHERE toDate(created_at) = '{date}'
        
    """
    ch.query_run(query_text)
    print(date)
# ch.query_run(query_text)

2025-05-21
2025-05-20
2025-05-19
2025-05-18
2025-05-17
2025-05-16
2025-05-15
2025-05-14
2025-05-13
2025-05-12
2025-05-11
2025-05-10
2025-05-09
2025-05-08
2025-05-07
2025-05-06
2025-05-05
2025-05-04
2025-05-03
2025-05-02
2025-05-01
2025-04-30
2025-04-29
2025-04-28
2025-04-27
2025-04-26
2025-04-25
2025-04-24
2025-04-23
2025-04-22
2025-04-21
2025-04-20
2025-04-19
2025-04-18
2025-04-17
2025-04-16
2025-04-15
2025-04-14
2025-04-13
2025-04-12
2025-04-11
2025-04-10
2025-04-09
2025-04-08
2025-04-07
2025-04-06
2025-04-05
2025-04-04
2025-04-03
2025-04-02
2025-04-01
2025-03-31
2025-03-30
2025-03-29
2025-03-28
2025-03-27
2025-03-26
2025-03-25
2025-03-24
2025-03-23
2025-03-22
2025-03-21
2025-03-20
2025-03-19
2025-03-18
2025-03-17
2025-03-16
2025-03-15
2025-03-14
2025-03-13
2025-03-12
2025-03-11
2025-03-10
2025-03-09
2025-03-08
2025-03-07
2025-03-06
2025-03-05
2025-03-04
2025-03-03
2025-03-02
2025-03-01
2025-02-28
2025-02-27
2025-02-26
2025-02-25
2025-02-24
2025-02-23
2025-02-22
2025-02-21
2025-02-20

In [21]:
query_text = """--sql
SELECT
   `billing_account_id`,
    `cost`,
    `count`,
    `created_at`,
    `device_type`,
    `device_uuid`,
    `partner_uuid`,
    toDate(created_at) AS `report_date`,
    `service`,
    `state`,
    `total`

FROM db1.billing_orders_devices_dir_partner
limit 10
    """

ch.query_run(query_text)

,billing_account_id,cost,count,created_at,device_type,device_uuid,partner_uuid,report_date,service,state,total
0,18,15.0,1,2025-01-24 00:00:09,camera,003d8f66-31ae-4561-92e5-2b21c7930a15,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-01-24,archive_3_kz,success,15.0
1,18,15.0,1,2025-01-24 00:00:09,intercom,029ee312-ea6f-4adb-a68a-b88d27a774aa,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-01-24,archive_3_kz,success,15.0
2,18,15.0,1,2025-01-24 00:00:10,intercom,03d24162-0350-44aa-94ec-76f04ecf37c8,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-01-24,archive_3_kz,success,15.0
3,18,15.0,1,2025-01-24 00:00:10,intercom,03f89b86-1303-451b-b0ec-6cfc408e8fe9,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-01-24,archive_3_kz,success,15.0
4,78,2.8,1,2025-01-24 00:00:10,intercom,707c6373-dba9-4e1e-b5ad-3bcd8f6edb2c,fdae9241-37f2-4446-866c-cd9119425fe4,2025-01-24,archive_3_pro,success,2.8
5,18,15.0,1,2025-01-24 00:00:10,intercom,048e2669-8ec5-48f8-a0eb-7145e032c0d8,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-01-24,archive_3_kz,success,15.0
6,18,15.0,1,2025-01-24 00:00:10,camera,055f53e8-2858-421a-b736-bc1cfb2231b5,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-01-24,archive_3_kz,success,15.0
7,78,2.8,1,2025-01-24 00:00:10,intercom,77794c63-4274-489b-a4a3-4f9447feaec5,fdae9241-37f2-4446-866c-cd9119425fe4,2025-01-24,archive_3_pro,success,2.8
8,18,15.0,1,2025-01-24 00:00:10,camera,05f95ce5-90a7-4c0d-a7a7-d3cca406cae5,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-01-24,archive_3_kz,success,15.0
9,18,15.0,1,2025-01-24 00:00:10,camera,0711124e-0f71-41f9-8d8c-c5a8e3c309af,af8b5bf4-0108-4a0c-acc2-096f94959fb3,2025-01-24,archive_3_kz,success,15.0


In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.billing_orders_devices_st_partner_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.billing_orders_devices_st_partner_ch AS
    SELECT
        *
    FROM db1.billing_orders_devices_st_partner
    """

ch.query_run(query_text)

___

In [10]:
query_text = """--sql
WITH full_table AS(
SELECT
    installation_point_st_partner.partner_uuid AS partner_uuid,
    installation_point_st_partner.installation_point_id  AS installation_point_id,
    sessions_st_mobile.citizen_id AS citizen_id,
    installation_point_st_partner.monetization AS monetization,
    subscriptions_st_mobile.state AS subscriptions_state,
    entries_st_mobile.ble_available AS ble_available,
    citizens_dir_mobile.activated_at AS activated_at,
    city
FROM db1.`sessions_st_mobile_ch` AS sessions_st_mobile
LEFT JOIN db1.subscriptions_st_mobile_ch AS subscriptions_st_mobile 
    ON subscriptions_st_mobile.citizen_id = sessions_st_mobile.citizen_id
    AND subscriptions_st_mobile.report_date = sessions_st_mobile.report_date
LEFT JOIN  db1.`citizens_st_mobile_ch` AS citizens_st_mobile 
    ON citizens_st_mobile.citizen_id = sessions_st_mobile.citizen_id
    AND citizens_st_mobile.report_date = sessions_st_mobile.report_date
LEFT JOIN db1.`citizens_dir_mobile_ch` AS citizens_dir_mobile ON citizens_dir_mobile.citizen_id = sessions_st_mobile.citizen_id
LEFT JOIN db1.`flats_dir_partner_ch` AS flats_dir_partner ON flats_dir_partner.flat_uuid = citizens_dir_mobile.flat_uuid
LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points_dir_partner ON flats_dir_partner.address_uuid = entries_installation_points_dir_partner.address_uuid
LEFT JOIN db1.`installation_point_st_partner_ch` AS installation_point_st_partner 
            ON installation_point_st_partner.installation_point_id = entries_installation_points_dir_partner.installation_point_id
            AND installation_point_st_partner.report_date = citizens_st_mobile.report_date
LEFT JOIN db1.`entries_st_mobile_ch` AS entries_st_mobile
            ON `entries_st_mobile`.`report_date` = sessions_st_mobile.`report_date` 
            AND`entries_st_mobile`.`address_uuid` = flats_dir_partner.`address_uuid`
WHERE DateTime::MakeDate(DateTime::ParseIso8601(`last_use`)) 
    BETWEEN '2025-06-01' and '2025-06-30'
    AND citizens_st_mobile.state = 'activated'
)
--
SELECT
    '2025-06-30' AS report_date,
    partner_uuid,
    city,
    count(DISTINCT citizen_id) AS total_active_users,
    count(DISTINCT if(DateTime::MakeDate(DateTime::ParseIso8601(activated_at)) 
                    BETWEEN '2025-06-01' and '2025-06-30',citizen_id,null)) new_active_users,
    count(DISTINCT if(monetization = 1,citizen_id,null)) as total_active_users_monetization,
    count(DISTINCT if(monetization = 1 and subscriptions_state = 'activated',citizen_id,null)) as total_active_user_subscribed_monetization,
    count(DISTINCT if(ble_available = 'true',citizen_id)) AS total_active_users_ble_available,
    count(DISTINCT if(ble_available = 'true' and monetization = 1,citizen_id)) AS total_active_users_ble_available_monetization,
    count(DISTINCT if(ble_available = 'true' and monetization = 1 and subscriptions_state = 'activated',citizen_id)) AS total_active_users_ble_available_subscribed_monetization
FROM full_table
GROUP BY
    partner_uuid,
    city
LIMIT 10
    """

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 62
 Code: 62. DB::Exception: Syntax error: failed at position 1953 (::) (line 28, col 34): ::ParseIso8601(`last_use`)) 
    BETWEEN '2025-06-01' and '2025-06-30'
    AND citizens_st_mobile.state = 'activated'
)
--
SELECT
    '2025-06-30' AS report_date,
    ... Expected one of: list, delimited by binary operators, literal, NULL, NULL, number, Bool, TRUE, FALSE, string literal, combination of maps, arrays, tuples, data type, identifier. (SYNTAX_ERROR) (version 25.3.3.42 (official build))
